https://learn.deeplearning.ai/langchain-chat-with-your-data

In [1]:
import openai
import config
openai.api_key = config.OPENAI_KEY


#### Loading and Chunking Input Documents

In [2]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

directory = DirectoryLoader(
    "docs/", glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader
)
pages = directory.load()

100%|██████████| 42/42 [00:22<00:00,  1.86it/s]


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 400
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    separators= ["\n\n", "\n", "(?<=\. )", " ", ]
)

docs = r_splitter.split_documents(pages)
len(docs)

2528

#### Embedding and VectorDB

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(openai_api_key=config.OPENAI_KEY)
persist_directory = "chroma/"

In [5]:
# new db from documents

vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

vectordb.persist()

#### Retrieving relevant chunks

In [6]:
# reopen existing db

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [17]:
question = "Was macht Alexandre?"
# "Was macht Alexandre?" 
# "Was würde Alf auf die Frage 'wie geht es dir?' antworten?"

In [ ]:
docs = vectordb.similarity_search(question, k=10)

In [18]:
docs = vectordb.max_marginal_relevance_search(question, fetch_k=20, k=10)

In [10]:
print(docs)

[Document(page_content='telefonischen Nachfragen, die wir beide bekommen haben. \nZum Schluss und zum wirklich guten Ende, brachte mich mein ewig-hilfreicher Vetter \nHeiri nach Zurzach, um Alf in seinem bequemen Wagen abzuholen. Nun werdet Ihr alle \nverstehen, dass wir uns zusammen zu Hause so wohlfühlen und gerne Rückschau halten \nauf das vergangene Jahr!', metadata={'page': 1, 'source': 'docs/891100-Weihnachtsbrief.pdf'}), Document(page_content='Tag die treppe auf und ab in sein Zimmer oder in den Keller. Er geniesst das \nschone Wetter für einstündige Spaziergange, und das wichtigste ist, dass er gar \nkeine Schmerzen mehr spürt. So arbeitet er weiter an seinen Memoiren in Ruhe und \nZufriedenheit. \nEs war geplant, dass ich Alf jeden zweiten Tag besuchen würde im Spital um ihm \ndie Zeit zu verkürzen, jedoch ... es kam alles anders ...', metadata={'page': 0, 'source': 'docs/891100-Weihnachtsbrief.pdf'})]


#### Using GPT for question answering

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=config.OPENAI_KEY, model_name="gpt-4", temperature=0)

In [19]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": question})
result["result"]

'Alexander ist ein Schüler, der sich in der Sekundarschule gut eingelebt hat und Querflöte spielt. Er ist ein großer Militärfan und sammelt leidenschaftlich Ausrüstungsgegenstände aus Militärbeständen. In seiner Freizeit fährt er gerne Rollbrett und ist am Snowboarden und Felsklettern interessiert.'

#### Using GPT for task generation

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Erzeuge eine Single-Choice-Aufgabe mit 4 Antwortoptionen, wovon eine richtig ist. Die Aufgabe soll folgendes Wissen prüfen: 
{context}
Single-Choice-Aufgabe:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Baltimore"
result = qa_chain({"query": question})
result["source_documents"]

In [ ]:
result["result"]